## Work bench to improve parsing and updating laws
This notebook is targeted to "1928399.pdf" and the change to the Zivilprozessordnung.

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lawinprogress.apply_changes.apply_changes import apply_changes
from lawinprogress.parsing.change_law_utils import preprocess_raw_law
from lawinprogress.parsing.parse_change_law import (
    parse_change_request_line,
    parse_change_law_tree,
)
from lawinprogress.parsing.parse_source_law import parse_source_law_tree
from lawinprogress.parsing.lawtree import LawTextNode
from lawinprogress.parsing.proposal_pdf_to_artikles import (
    extract_law_titles,
    extract_raw_proposal,
    extract_seperate_change_proposals,
    read_pdf_law,
    remove_inkrafttreten,
)

### Read and process the change law

In [3]:
change_law_path = "../data/1928399.pdf"

In [4]:
change_law_raw = read_pdf_law(change_law_path)

# idenfify the different laws affected
change_law_extract = extract_raw_proposal(change_law_raw)
proposals_list = extract_seperate_change_proposals(change_law_extract)
law_titles = extract_law_titles(proposals_list)
law_titles, proposals_list = remove_inkrafttreten(law_titles, proposals_list)

In [5]:
law_titles

['Zivilprozessordnung',
 'Strafprozessordnung',
 'Gesetzes  über  das Verfahren  in Familiensachen und in  Angelegenheiten der   freiwilligen Gerichtsbarkeit',
 'Elektronischer-Rechtsverkehr-Verordnung',
 'Arbeitsgerichtsgesetzes',
 'Arbeitsgerichtsgesetzes',
 'Arbeitsgerichtsgesetzes zum  1. Januar 2022',
 'Arbeitsgerichtsgesetzes zum  1. Januar 2026',
 'Sozialgerichtsgesetzes',
 'Sozialgerichtsgesetzes zum  1. Januar 2022',
 'Sozialgerichtsgesetzes zum  1. Januar 2026',
 'Verwaltungsgerichtsordnung',
 'Verwaltungsgerichtsordnung zum  1. Januar 2022',
 'Verwaltungsgerichtsordnung zum  1. Januar 2026',
 'Finanzgerichtsordnung',
 'Finanzgerichtsordnung zum  1. Januar 2022',
 'Finanzgerichtsordnung zum  1. Januar 2026',
 'Bundesrechtsanwaltsordnung',
 'Beurkundungsgesetzes',
 'Gesetzes  über die  Tätigkeit  europäischer Rechtsanwälte in Deutschland',
 'Verordnung zur Einführung von Vordrucken für das Mahnverfahren',
 'Zustellungsvordruckverordnung',
 'Strafvollzugsgesetzes',
 'Grundbucho

In [6]:
# we only work with the first one here
law_index = 4
change_law = proposals_list[law_index]
print(change_law)

Änderung des  Arbeitsgerichtsgesetzes  
Das  Arbeitsgerichtsgesetz in der  Fassung  der  Bekanntmachung  vom  2. Juli  1979  (BGBl. I  S. 853, 1036),  das  
zuletzt  durch Artikel 12 des Gesetzes  vom  12.  Juni  2020  (BGBl. I  S. 1248)  geändert  worden ist, wird wie folgt  
geändert:  
1.  In § 64 Absatz 7  werden  die Wörter  „Die Vorschriften  des  §“ durch die Wörter  „Die  Vorschriften der  §§ 46c  
bis 46f,“ ersetzt  und  werden nach den Wörtern „und der §§ 62  und 63 über“  die Wörter „den elektronischen  
Rechtsverkehr,“ eingefügt.  
2.  In § 72 Absatz 6  werden  die Wörter  „Die Vorschriften  des  §“ durch die Wörter  „Die  Vorschriften der  §§ 46c  
bis  46f,“  ersetzt  und werden  nach  den Wörtern „und des  § 63  dieses  Gesetzes  über“  die Wörter  „den  elekt-
ronischen  Rechtsverkehr,“  eingefügt.  
3.  § 80 Absatz 2  Satz 1 wird wie folgt gefasst:   
„Für  das  Beschlussverfahren des  ersten  Rechtszugs  gelten  die für  das  Urteilsverfahren des  ersten Rechtszugs  


In [7]:
# parse source law
clean_change_law = preprocess_raw_law(change_law)
parsed_change_law_tree = LawTextNode(text=law_titles[law_index], bulletpoint="change")
parsed_change_law_tree = parse_change_law_tree(
    text=clean_change_law, source_node=parsed_change_law_tree
)

In [8]:
parsed_change_law_tree._print()

change - Arbeitsgerichtsgesetzes
├── 1. - In § 64 Absatz 7  werden  die Wörter  „Die Vorschriften  des  §“ durch die Wörter  „Die  Vorschriften der  §§ 46c   bis 46f,“ ersetzt  und  werden nach den Wörtern „und der §§ 62  und 63 über“  die Wörter „den elektronischen   Rechtsverkehr,“ eingefügt.
├── 2. - In § 72 Absatz 6  werden  die Wörter  „Die Vorschriften  des  §“ durch die Wörter  „Die  Vorschriften der  §§ 46c   bis  46f,“  ersetzt  und werden  nach  den Wörtern „und des  § 63  dieses  Gesetzes  über“  die Wörter  „den  elektronischen  Rechtsverkehr,“  eingefügt.
├── 3. - § 80 Absatz 2  Satz 1 wird wie folgt gefasst:„Für  das  Beschlussverfahren des  ersten  Rechtszugs  gelten  die für  das  Urteilsverfahren des  ersten Rechtszugs   maßgebenden Vorschriften entsprechend, soweit sich aus  den §§ 81 bis 84 nichts anderes ergibt.“
├── 4. - § 87 Absatz 2  Satz 1 wird wie folgt gefasst:„Für  das  Beschwerdeverfahren gelten  die für  das Berufungsverfahren  maßgebenden  Vorschriften sow

In [9]:
# build up a list of texts that describe a single change
all_change_lines = []
for l in parsed_change_law_tree.leaves:
    path = [str(l)]
    node = l
    while node.parent:
        node = node.parent
        path.append(str(node))
    change_line = " ".join(path[::-1][1:])
    all_change_lines.append(change_line)

In [10]:
# parse the change requests in a structured format
change_requests = []
for change_request_line in all_change_lines:
    res = parse_change_request_line(change_request_line)
    if res:
        change_requests.extend(res)

In [11]:
change_requests

[Change(location=['§ 64', 'Absatz 7'], sentences=[], text=['Die Vorschriften  des  §', 'Die  Vorschriften der  §§ 46c   bis 46f,', 'und der §§ 62  und 63 über', 'den elektronischen   Rechtsverkehr,'], change_type='MULTIPLE_CHANGES'),
 Change(location=['§ 72', 'Absatz 6'], sentences=[], text=['Die Vorschriften  des  §', 'Die  Vorschriften der  §§ 46c   bis  46f,', 'und des  § 63  dieses  Gesetzes  über', 'den  elektronischen  Rechtsverkehr,'], change_type='MULTIPLE_CHANGES'),
 Change(location=['§ 80', 'Absatz 2'], sentences=['Satz 1'], text=['Für  das  Beschlussverfahren des  ersten  Rechtszugs  gelten  die für  das  Urteilsverfahren des  ersten Rechtszugs   maßgebenden Vorschriften entsprechend, soweit sich aus  den §§ 81 bis 84 nichts anderes ergibt.'], change_type='rephrase'),
 Change(location=['§ 87', 'Absatz 2'], sentences=['Satz 1'], text=['Für  das  Beschwerdeverfahren gelten  die für  das Berufungsverfahren  maßgebenden  Vorschriften sowie die   Vorschrift  des  §  85 über  die 

### Process the source law

In [12]:
law_name = "Arbeitsgerichtsgesetz"  # "Strafprozessordnung"  # "Zivilprozessordnung"
source_law_path = "../data/source_laws/{}.txt".format(law_name)

In [13]:
# read the source law
with open(source_law_path, "r") as file:
    source_law_text = file.read()

In [26]:
# parse source law
parsed_law_tree = LawTextNode(text=law_name, bulletpoint="source")
parsed_law_tree = parse_source_law_tree(
    text=source_law_text, source_node=parsed_law_tree
)

In [27]:
parsed_law_tree._print()

source - Arbeitsgerichtsgesetz
├── § 1 - Gerichte für Arbeitssachen
Die Gerichtsbarkeit in Arbeitssachen - §§ 2 bis 3 - wird ausgeübt durch die Arbeitsgerichte - §§ 14 bis 31 -, die Landesarbeitsgerichte - §§ 33 bis 39 - und das Bundesarbeitsgericht - §§ 40 bis 45 - (Gerichte für Arbeitssachen).
Nichtamtliches Inhaltsverzeichnis
├── § 2 - Zuständigkeit im Urteilsverfahren
│   ├── (1) - Die Gerichte für Arbeitssachen sind ausschließlich zuständig für
│   │   ├── 1. - bürgerliche Rechtsstreitigkeiten zwischen Tarifvertragsparteien oder zwischen diesen und Dritten aus Tarifverträgen oder über das Bestehen oder Nichtbestehen von Tarifverträgen;
│   │   ├── 2. - bürgerliche Rechtsstreitigkeiten zwischen tariffähigen Parteien oder zwischen diesen und Dritten aus unerlaubten Handlungen, soweit es sich um Maßnahmen zum Zwecke des Arbeitskampfs oder um Fragen der Vereinigungsfreiheit einschließlich des hiermit im Zusammenhang stehenden Betätigungsrechts der Vereinigungen handelt;
│   │   ├── 3.

In [28]:
print(parsed_law_tree.to_text())

source Arbeitsgerichtsgesetz
    § 1 Gerichte für Arbeitssachen
Die Gerichtsbarkeit in Arbeitssachen - §§ 2 bis 3 - wird ausgeübt durch die Arbeitsgerichte - §§ 14 bis 31 -, die Landesarbeitsgerichte - §§ 33 bis 39 - und das Bundesarbeitsgericht - §§ 40 bis 45 - (Gerichte für Arbeitssachen).
Nichtamtliches Inhaltsverzeichnis
    § 2 Zuständigkeit im Urteilsverfahren
        (1) Die Gerichte für Arbeitssachen sind ausschließlich zuständig für
            1. bürgerliche Rechtsstreitigkeiten zwischen Tarifvertragsparteien oder zwischen diesen und Dritten aus Tarifverträgen oder über das Bestehen oder Nichtbestehen von Tarifverträgen;
            2. bürgerliche Rechtsstreitigkeiten zwischen tariffähigen Parteien oder zwischen diesen und Dritten aus unerlaubten Handlungen, soweit es sich um Maßnahmen zum Zwecke des Arbeitskampfs oder um Fragen der Vereinigungsfreiheit einschließlich des hiermit im Zusammenhang stehenden Betätigungsrechts der Vereinigungen handelt;
            3. bürgerliche

### Update the source law

In [29]:
res_law_tree = apply_changes(parsed_law_tree, change_requests)

SKIPPED MULTIPLE_CHANGES: Change(location=['§ 64', 'Absatz 7'], sentences=[], text=['Die Vorschriften  des  §', 'Die  Vorschriften der  §§ 46c   bis 46f,', 'und der §§ 62  und 63 über', 'den elektronischen   Rechtsverkehr,'], change_type='MULTIPLE_CHANGES')
SKIPPED MULTIPLE_CHANGES: Change(location=['§ 72', 'Absatz 6'], sentences=[], text=['Die Vorschriften  des  §', 'Die  Vorschriften der  §§ 46c   bis  46f,', 'und des  § 63  dieses  Gesetzes  über', 'den  elektronischen  Rechtsverkehr,'], change_type='MULTIPLE_CHANGES')
APPLIED rephrase:
Change(location=['§ 80', 'Absatz 2'], sentences=['Satz 1'], text=['Für  das  Beschlussverfahren des  ersten  Rechtszugs  gelten  die für  das  Urteilsverfahren des  ersten Rechtszugs   maßgebenden Vorschriften entsprechend, soweit sich aus  den §§ 81 bis 84 nichts anderes ergibt.'], change_type='rephrase')
APPLIED rephrase:
Change(location=['§ 87', 'Absatz 2'], sentences=['Satz 1'], text=['Für  das  Beschwerdeverfahren gelten  die für  das Berufungsv

In [18]:
print(res_law_tree.to_text())

source Arbeitsgerichtsgesetz
    § 1 Gerichte für Arbeitssachen
Die Gerichtsbarkeit in Arbeitssachen - §§ 2 bis 3 - wird ausgeübt durch die Arbeitsgerichte - §§ 14 bis 31 -, die Landesarbeitsgerichte - §§ 33 bis 39 - und das Bundesarbeitsgericht - §§ 40 bis 45 - (Gerichte für Arbeitssachen).
Nichtamtliches Inhaltsverzeichnis
    § 2 Zuständigkeit im Urteilsverfahren
        (1) Die Gerichte für Arbeitssachen sind ausschließlich zuständig für
            1. bürgerliche Rechtsstreitigkeiten zwischen Tarifvertragsparteien oder zwischen diesen und Dritten aus Tarifverträgen oder über das Bestehen oder Nichtbestehen von Tarifverträgen;
            2. bürgerliche Rechtsstreitigkeiten zwischen tariffähigen Parteien oder zwischen diesen und Dritten aus unerlaubten Handlungen, soweit es sich um Maßnahmen zum Zwecke des Arbeitskampfs oder um Fragen der Vereinigungsfreiheit einschließlich des hiermit im Zusammenhang stehenden Betätigungsrechts der Vereinigungen handelt;
            3. bürgerliche